# Aritmética modular

## Ejercicio 1

Implementa el algoritmo extendido de Euclides para el cálculo del máximo común divisor: dados dos enteros $a$ y $b$, encuentra $u, v ∈ \mathbb{Z}$ tales que $au + bv$ es el máximo común divisor de $a$ y $b$.

In [1]:
def ext_euclides(a, b):
    if b == 0:
        return a, 1, 0
    else:
        u1, u2, v1, v2 = 0, 1, 1, 0
        
        while b > 0: # Mientras que b sea mayor que 0
            q = a // b                                     # a = |_ a / b _|
            r, u, v = a - q * b, u2 - q * u1, v2 - q * v1  # r = a - qb, u = u2 - qu1, v = v2 -qv1
            a, b, u2, u1, v2, v1 = b, r, u1, u, v1, v      
            
        return a, u2, v2
    

d, u, v = ext_euclides(4864, 3458)
print("d = ", d, " u = ", u, " v = ", v)

d =  38  u =  32  v =  -45


En el código podemos ver como la función `ext_euclides` recibe como parámetros de entrada dos enteros $a$ y $b$ y devuelve el máximo común divisor, seguidos por $u$ y $v$. 

La función sigue el ejemplo de código del algoritmo _2.107_ de [A. Menezes, P. van Oorschot, and S. Vanstone, Handbook of Applied Cryptography, CRC Press, 1996.](http://cacr.uwaterloo.ca/hac/about/chap2.pdf)

## Ejercicio 2

Usando el ejercicio anterior, escribe una función que calcule $a^{-1} \bmod b$ para cualesquiera $a, b$ enteros que sean primos relativos.

In [2]:
def inverse(a,b):
    return ext_euclides(a,b)[1]


inverse(2, 5)

-2

A partir del código del ejercicio 1, en caso de que exista inversa en $\mathbb{Z}_n$, obtendremos lo siguiente: $$d = au + bv$$ En caso de que $a$ tenga inversa en $\mathbb{Z}_n$, tendremos que $\text{mcd}(a,n) = 1$. Por tanto, por la identidad de Bezout, tenemos que existen $u$ y $v$ (coeficientes de Bezout) tal que: $$1 = ua + vn$$

Por tanto, si estamos en el espacio $\mathbb{Z}_n$, tenemos que $$ \begin{matrix}1 = ua + vn & =& ua + 0 \\ & \Rightarrow & ua & a \in \mathcal{U}(\mathbb{Z}_n)\\  u & = & a^{-1} \end{matrix}$$

Para devolver el inverso correcto, devolveremos $u \bmod n$

## Ejercicio 3

Escribe una función que calcule $a^b \bmod n$ para cualesquiera $a, b\text{ y } n$. La implementación debe tener en cuenta la representación binaria de $b$.

In [3]:
def big_pow(a, b, n):
    if b == 1:
        return a % n
    
    if a == 1:
        return 1
    
    a0, b0, p = a, b, 1
    
    while b0 > 0:
        # Si el bit está a 1, se incrementa el valor del producto
        if b0 % 2 == 1:
            p = p * a0
            
        b0, a0 = b0 // 2, a0*a0%n
            
    return p % n


big_pow(156187561565735418, 43498489489156978415674, 23)

16

Para calcular $a^b \bmod n$, podemos tomar como base que, el exponente $b$ puede escribirse en binario como $b = b_0b_1\ldots b_k$ tal que $b_i = 0\;|\;1$. A partir de esto, podemos definir $b$ como $b = \sum_{i=0}^k b_i\cdot2^i$.

Entonces, la expresión $a^b$ se puede expresar como: $$a^b = a^{\sum_{i=0}^k b_i\cdot2^i} = \prod a^{b_i2^i} = \prod \left(a ^{2^i}\right)^{b_i}$$

Con esto podemos ver que el valor del producto se incrementará cuando el valor de $b_i = 1$, elevándose el valor del producto al cuadrado.

Una forma de realizar este cálculo, es la que aparece en el código, y es ir realizando las operaciones $b_i = b \bmod 2; \quad b = \lfloor b | 2 \rfloor$, e ir siempre incrementando el valor de $a$ como $a = a^2 \bmod n$. El valor del producto, denotado como $p$, se incrementará cuando el valor de $b = 1$, tal que $p = (p \cdot a)\bmod n$.

## Ejercicio 4

Dado un entero $p$, escribe una función para determinar si $p$ es probablemente primo usando el método de Miller-Rabin.

In [4]:
from random import randint
from functools import reduce

def bifactor(num):
    a0, s = num, 0
    while a0 > 0 and a0 % 2 == 0:
        a0 //= 2
        s += 1
    return s, a0


def miller_rabin(num):
    # primos menores que 5
    if num == 2 or num == 3:
        return True
    # es 4 o menor que 4 o es par mayor que 2
    elif num == 4 or num < 2 or num % 2 == 0:
        return False
    else:
        s, u = bifactor(num - 1)   # Calculamos p-1 = 2^s * u
        a = randint(2, num - 2)    # a in [2, ..., p - 2]
        l = [big_pow(a, (2**i) * u, num) for i in range(s + 1)] # l = [a^u, a^2u, ..., a^2su] 
        # Primer elemento de la lista es 1 o -1
        if l[0] == 1 or l[0] == num - 1:
            return True # Probablemente primo

        # Ninguna de las potencias es igual a 1
        elif 1 not in l:
            return False # No es primo

        # Si aparece un 1 en la lista no precedido de un -1
        elif 1 in l and l[l.index(1)-1] != num - 1:
            return False # No primo

        # Si -1 está en la lista y no es el último elemento
        elif num - 1 in l and l[-1] != num - 1:
            return True  # Probablemente primo
    
    
def miller_rabin_test(num, n = 10):
    return reduce(lambda x, y: x and y, [miller_rabin(num) for i in range(n)])
    
print(miller_rabin(41)) # Primo conocido
print(miller_rabin_test(127973)) # Primo conocido
print(miller_rabin_test(127972)) # No primo
print(miller_rabin_test(123456789101119)) # Primo conocido
print(miller_rabin_test(28564333765949)) # Primo conocido


True
True
False
True
True


El test de Miller-Rabin lo realiza la función `miller_rabin`. Esta función se encarga de dado un "primo" $p$, se encarga de descomponer $p-1$ en $p-1 = 2^s\cdot u$. 

Con esto compone la lista $l = [a^{2^0u}, a^{2^0u}, \ldots, a^{2^su}]$, siendo $a \in_R [2, \ldots, p - 2]$ y pasamos a comprobar las condiciones de ser ___probablemente primo___ o ___no primo___:
* Si el primer elemento de $l$ es 1 o -1, $p$ será probablemente primo.
* Si no aparece 1 en $l$, es no primo, ya que no supera el test de Fermat.
* Si aparece un 1 no precedido de -1, es no primo, ya que existe un raíz cuadrada de 1 que no es ni 1 ni -1.
* Si aparece un -1 en $l$, es probablemente primo, ya que el siguiente elemento en la lista será 1.

## Ejercicio 5

Implementa el algoritmo paso enano-paso gigante para el cálculo de logaritmos discretos en $\mathbb{Z}_p$.

In [5]:
from math import sqrt, ceil
from time import time

def baby_pass_giant_pass(a, b, p):
    # Si p es primo
    if miller_rabin_test(p):
        # Buscamos k tal que a^k = b, con a,b in Z_p
        if b == 1:
            return 0 # k = 0
        
        else:
            # Si k existe -> k = cs -r; 0 <= r < s; 1 <= c <= s
            s = ceil(sqrt(p - 1))
            # giant pass
            L = [pow(a, i*s, p) for i in range(1, s + 1)]
            # baby pass
            l = [(b * a**i) % p for i in range(s)]
            # calculamos la intersección entre L y l
            ks = list(filter(lambda x: x in L, l))
            # calculamos los k, que en caso de que p
            # no sea primitivo, habrá varios k
            for k in ks:
                yield (L.index(k) + 1) * s - l.index(k)
    
    else:
        print("p =", p, "no es primo.")
    
    return None

list(baby_pass_giant_pass(6, 32, 41))

[10]

La función realiza el cálculo del algoritmo paso enano-paso gigante, comprobando primero que $p$ es primo. En caso de no serlo, devuelve un valor nulo. En caso de que sea primo, comprueba si $b = 1$ o no. Si lo es, devuelve $k = 0$ ya que $a^0 = 1$. En caso contrario pasamos a realizar el algoritmo paso enano-paso gigante:
* __Paso gigante__: calcularemos la lista $L$ como $L = [a^s, a^{2s}, \ldots, a^{ss}]$, donde en cada paso, se multiplica el valor anterior por $a^s$.
* __Paso enano__: calcularemos la lista $l$ como $l = [b, ba, \ldots, ba^{s-1}]$, donde en cada paso, multiplicamos el valor anterior por $a$.

Si $L \cap l \neq \emptyset$, existe al menos un $k$ tal que $a^k = b$ con $a,b \in \mathbb{Z}_p$. Además, tenemos que $$a^{cs}\in L \quad = \quad ba^r \in l$$ por lo tanto $k = cs - r$.

Este $k$ es el que se calcula en el bucle `for` de la función. Esta función devuelve un generador, que en Python 3 podemos pasar a lista con la función `list` o ir generándolos uno por uno.

## Ejercicio 6

Sea $n = pq$, con $p$ y $q$ enteros primos positivos.
* Escribe una función que, dado un entero $a$ y un primo $p$ con $\left(\frac{a}{p}\right) = 1$, devuelva $r$ tal que $r^2 \equiv a \bmod p$; primero te hará falta implentar el símbolo de Jacobi.
* Sea $a$ un entero que es residuo cuadrático módulo $p$ y $q$. Usa el teorema chino de los restos para calcular todas las raíces cuadradas de $a$ módulo $p$ y $q$.

In [6]:
def Jacobi(a, p):
    if p % 2 != 0:
        symbol = 1  # inicializamos el símbolo de jacobi
        a0 = a % p  # 1: aplicamos (a / p) = (a % p / p)

        if a0 == 0:
            return 0
        elif a0 == 1:
            return 1
        elif a0 == -1:  # 5: -1 / p) = -1
            return ((-1) ** ((p - 1) // 2))

        u, s = bifactor(a0)  # 2: (ab / p) = (a / p)*(b / p)

        if u > 0:  # 3: (2 / p)  = (-1)**((p^2 - 1)/8)
            symbol = ((-1) ** ((p ** 2 - 1) // 8))**u

        # se puede descomponer n en a * b
        # y son distintos de 1 y -1
        # y p es impar
        if s == 1:  # 4: (1 / p)  = 1
            return symbol

        elif s == -1:  # 5: -1 / p) = -1
            return symbol * ((-1) ** ((p - 1) // 2))

        elif p % 2 != 0:  # 6: (q / p)  = (-1)**((p - 1)(q - 1)/4) * (p / q)
            return symbol * Jacobi(p, s) * (-1) ** ((p - 1) * (s - 1) // 4)
    else:
        return None

Jacobi(4, 1009)

1

En esta función implementamos el símbolo de Jacobi. Esta implementación, aplica las siguientes reglas, prácticamente en el mismo orden de aparición:
1. $\left(\frac{a}{p}\right) = \left(\frac{a \bmod p}{p}\right)$
2. $\left(\frac{ab}{p}\right) = \left(\frac{a}{p}\right)\left(\frac{b}{p}\right)$. Esta regla, en particular, lo que hacemos es descomponer el número $n$ en $n = 2^su$, por tanto, esto pasa a ser $$\left(\frac{n}{p}\right) = \left(\frac{2^u}{p}\right)\left(\frac{s}{p}\right)$$
3. $\left(\frac{2}{p}\right) = (-1)^\frac{p^2 - 1}{8}$
4. $\left(\frac{1}{p}\right) = 1$
5. $\left(\frac{1}{p}\right) = (-1)^\frac{p-1}{2}$
6. $\left(\frac{q}{p}\right)$, si $p$ y $q$ son ambos impares, hacemos lo siguiente: $$ \left(\frac{q}{p}\right) = (-1)^\frac{(p - 1)(q - 1)}{4}\left(\frac{p}{q}\right) =  \begin{cases}
     - \left(\frac{p}{q}\right) & \quad \text{si } p \equiv q \equiv 3 \bmod 4 \\
    \left(\frac{p}{q}\right) & \quad \text{en caso contrario}\\
  \end{cases}$$
  
Esta función calcula el símbolo de Jacobi de forma recursiva, pero no es una recursividad tan fuerte ya que, al descomponer $n$ en $n = 2^su$, aplicamos la regla 3, calculando el símbolo de la primera parte, y ya nos centramos en calcular el símbolo de $\left(\frac{u}{p}\right)$. A continuación, vamos a usar esto para calcular la raíz modular.

In [7]:
def sqrt_mod(a, p):
    # si el número tiene raíz en Z_p
    if Jacobi(a, p) == 1:
        # buscamos un n tal que (n / p) = -1, es decir
        # n no es un residuo cuadrático
        for n in range(2, p):
            if Jacobi(n, p) == -1:
                break
        else:
            print("Error: no tiene inverso")
            return None
        
        # Descomponemos p - 1 en 2^su
        u, s = bifactor(p - 1)
        # si u = 1, hacemos lo siguiente
        if u == 1:
            return big_pow(a, ((p+1) // 4), p)
        # en si u >= 2
        elif u >= 2:
            r, b, j, inv_a = big_pow(a, ((s+1) // 2), p), big_pow(n, s, p), 0, inverse(a, p)
            while j <= u - 2:
                if big_pow(inv_a*r**2, 2**(u - 2 - j), p) == p-1:
                    r *=b 
                    r %= p
                    
                b = b**2 
                j+=1
            
            return r
    else:
        print("No tiene raíz cuadrada módulo", p)
        return None

# Devuelve las 2 raíces del número a en Z_p
def sqrts_mod(a, p):
    sqrt = sqrt_mod(a, p)
    sol = [sqrt, p - sqrt]
    sol.sort()
    return sol

sqrts_mod(64, 1009)

[8, 1001]

Una vez que tenemos las raíces en un número $a$ en un cuerpo $\mathbb{Z}_p$, podemos usar el ___Teorema Chino de los Restos___ para hallar las raíces de un número $a$ en $\mathbb{Z}_n$, donde $n = p\cdot q$. Para ello, usaremos las siguientes funciones:

In [17]:
def norm(cong):
    x, y, m = cong # separamos los coeficientes de la congruencia
    d, s = ext_euclides(x, m)[0:2] # obtenemos el mcd(x, m)
    # Si y mod d es distinto de 0, la congruencia no tiene solución
    if y % d != 0: 
        raise "Error: congruencia sin solución"
    # en caso contrario, normalizamos la congruencia
    else:
        # h = |_ y / d _|, f = |_ m / d _|
        h, f = y // d, m // d
        e = (h * s) % f
        return [1, e, f,]
    

def chinese_remainder(cong1, cong2, n):
    # cong1 => ax  = b  mod p => norm(cong1) => x = a mod p
    # cong2 => a'x = b' mod q => norm(cong2) => x = b mod q
    x1, a, p = norm(cong1)
    x2, b, q = norm(cong2)
    
    inv_p = inverse(p, q)
    aux = (b-a)*inv_p
    
    return (a + p*aux)%n

chinese_remainder([1,12,17], [1, 34, 41], 697)

403